In [1]:
pwd

'/home/rnshishir/deepmed/CPRD_Cut22/debug/clean'

In [2]:
import pyspark

In [3]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.functions import merge
from utils.utils import save_obj
from CPRD.functions.MedicalDictionary import * 
from CPRD.functions.Prediction import * 
from CPRD.functions.cohort_select_causal import * 
from CPRD.functions.SurvPrediction import * 
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [5]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'referral': '/home/workspace/datasets/cprd/cprd2021/*/*_Referral_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.

# Association to study: effect of antihypertensives on ischaemic heart diseases
### cohort selection
-Cohort selection: age between 60 and 61 years in years between 2009 and 2010

-take the first initiation of antihyp in this time to be baseline

-take random baseline for those without any antihyp

In [6]:
md = MedicalDictionaryRiskPrediction(file, spark)


In [9]:
md.findItem('')

['height',
 'weight',
 'bmi',
 'creatinine',
 'egfr',
 'hb',
 'hba1c',
 'smoking+yes',
 'smoking+ex',
 'heart rate',
 'dbp',
 'sbp',
 'smoking+no',
 'smoking+check',
 'cholesterol',
 'coronary heart disease family history',
 'new york heart association heart failure classification 1',
 'new york heart association heart failure classification 2',
 'new york heart association heart failure classification 3',
 'new york heart association heart failure classification 4',
 'new york heart association heart failure classification',
 'atrial fibrillation',
 'stroke',
 'transient ischaemic attack',
 'arterial thromboembolism',
 'aortic plaque',
 'diabetes',
 'hypertension',
 'heart failure',
 'gastrointestinal and intracranial bleeding',
 'gastrointestinal tract ulcer',
 'brain injury',
 'spinal injury',
 'eye injury',
 'dementia',
 'cardivascular mortality',
 'ischaemic stroke',
 'venous thromoemoblism and pulmonary embolism',
 'thromboembolic',
 'myocardial infarction',
 'vascular dementia',

In [10]:
tempantihyp = md.queryMedication(md.findItem('antihy'), merge= True)['merged']


In [13]:
# medical dict can give us both exposures and outcomes codes - e.g. diabetes as outcomes or antihyyp as exposurea
md = MedicalDictionaryRiskPrediction(file, spark)
tempantihyp = md.queryMedication(md.findItem('antihy'), merge= True)['merged']
expcodes = {}
expcodes['prodcode'] = tempantihyp['prodcode']


exposure selection

In [14]:
# CohortSoftCut from the causal cohort selection package has everything to select cohort and baseline
# specifically the baseline for those WITH the exposure is date of exposure, and for those WITHOUT exp of interest is random sampling of baseline 
cohortSelector = CohortSoftCut(1,60, 61,  expcodes, False, False, True)



In [15]:
# no mapping as you don't want to drop the prodcodes which are not mapped...
# medications = retrieve_medications(file, spark, mapping='none', duration=(2009, 2010), demographics=cohort, practiceLink=True)
# medications.write.parquet('/home/shared/shishir/AurumOut/rawDat/meds_nomapping_2009_2010_association_example.parquet')

medications = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/meds_nomapping_2009_2010_association_example.parquet')
# medications.select('patid').dropDuplicates().count()  - 208603 patients have meds in the time period
# medications.select('patid').count() - 11764962 number of records

# pipeline() function has 3 components:
1) demo extract gets eligible patients between age 60 and 61 ^ defined above and years 2009 and 2010  
2) extraction of the exposure of interest -  set  baseline as the date of the exposure first captured  
3) for those without exposure (i.e. control patients) , set up baseline as randomised baseline


In [14]:
cohort = cohortSelector.pipeline(file, spark, duration=('2009-01-01', '2010-01-01'), randomNeg=True, sourceT=medications, sourceCol='prodcode')

cohort.write.parquet('/home/shared/shishir/AurumOut/rawDat/cohort_association_example3.parquet')


In [17]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/cohort_association_example3.parquet')
cohort.count()

# 225564 pats 


225564

outcome selection

In [19]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/cohort_association_example3.parquet')

necessaryColumns = ['patid',
 'gender',
 'dob',
 'study_entry',
 'startdate',
 'enddate',
  'exp_label']

cohort = cohort.select(necessaryColumns)


In [20]:
cohort.show()

+-------------+------+----------+-----------+----------+----------+---------+
|        patid|gender|       dob|study_entry| startdate|   enddate|exp_label|
+-------------+------+----------+-----------+----------+----------+---------+
|1005150820614|     1|1948-07-01| 2009-05-22|2000-02-08|2016-03-15|        0|
|1012900720626|     2|1949-07-01| 2009-08-03|2004-03-11|2021-01-15|        0|
|1017091020359|     2|1949-07-01| 2009-07-25|1988-02-01|2013-02-05|        0|
|1021610620622|     1|1948-07-01| 2009-05-21|1978-04-15|2021-01-15|        0|
|1025015720649|     1|1948-07-01| 2009-05-03|1999-01-19|2015-11-30|        0|
| 102635020244|     2|1949-07-01| 2009-10-04|1990-10-26|2021-01-15|        0|
|1031916720613|     1|1949-07-01| 2009-10-09|1994-04-20|2021-01-05|        0|
|1031985420613|     1|1948-07-01| 2009-06-13|1968-06-25|2016-11-19|        0|
|1033364020631|     2|1949-07-01| 2009-11-04|1982-06-24|2021-01-05|        0|
|1033532820631|     1|1948-07-01| 2009-04-21|1997-08-12|2021-01-

In [21]:
# for sake of tutorial, sample .2% of cohort.
cohort = cohort.sample(withReplacement=False,fraction=0.002)


In [22]:
cohort.count()

441

outcome selection

In [25]:
# for looking at groups of conditions...
# md.findItem('ischaem') + md.findItem('heart failure') 


In [26]:
md.findItem('ischaem')

['transient ischaemic attack', 'ischaemic stroke', 'ischaemic heart disease']

In [27]:
# label codes phenotyping from medical dict - ie maybe ischaemic conditions
labelcodes  = md.queryDisease(md.findItem('ischaem'), merge= True)['merged']
allIschaemiaCodes = labelcodes['medcode'] + labelcodes['ICD10'] 

# can add OPCS procedure codes too for this, but for simplicity just looking at diagnosis here...:  + labelcodes['OPCS'] 



In [29]:
# label codes phenotyping from medical dict - ie maybe ischaemic conditions
allDiag = split_combine_diag (spark , '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
# read death registry as death is an important data source for looking for outcome  
death = tables.retrieve_death(dir=file['death'], spark=spark)

In [78]:
cohort.count()

470

In [31]:
survival_outcome_selector = SurvRiskPredictionBase(follow_up_duration_month=60)
# pipeline for capturing the event and time2event is below
rcohort = survival_outcome_selector.setupEventAndTime(cohort, source=allDiag, condition=allIschaemiaCodes, column='code', death=death)

rcohort. write.parquet('test10_40m.parquet')

In [80]:
ss = pd.read_parquet('test9_40m.parquet')

In [93]:
ss

,patid,gender,dob,study_entry,startdate,enddate,exp_label,eventdate,label,time2event
0,2764393220207,1,1949-07-01,2009-08-12,1998-07-02,2013-06-29,0,None,0,40
1,6099362721534,2,1948-07-01,2009-01-03,2004-01-28,2021-01-06,0,None,0,40
2,3271803221216,1,1948-07-01,2009-01-16,2005-12-14,2021-01-08,1,None,0,40
3,6562006120818,1,1948-07-01,2009-01-09,2006-12-18,2013-05-17,1,None,0,40
4,2411820820942,1,1949-07-01,2009-07-10,1991-03-31,2021-01-15,0,None,0,40
...,...,...,...,...,...,...,...,...,...,...
434,2433705820944,2,1948-07-01,2009-01-06,2006-08-04,2010-08-06,1,2010-01-08,1,12
435,3084540721103,1,1949-07-01,2009-09-02,2008-07-29,2016-01-15,0,2012-06-15,1,33
436,2183944520585,1,1949-07-01,2009-07-06,1986-07-31,2013-02-28,1,2009-07-20,1,0
437,2078103720892,1,1948-07-01,2009-03-19,1990-02-15,2012-07-10,0,2012-07-10,1,40


# T-BEHRT/BEHRT/TRisk etc data extraction 
### also option to make unique per N months (i.e., delete duplicate rec in a given N months)

In [ ]:
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')
proc = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/proc_HES_1985_2021_allpats.parquet').withColumnRenamed('OPCS','code')

proc = proc.withColumn('code', F.concat(F.lit("PROC_"), F.col('code'))).select('patid','eventdate','code')

allDiag = allDiag.select('patid','eventdate','ICD').withColumnRenamed('ICD','code')
meds = meds.select('patid','eventdate','code')
fullcodes = allDiag.union(meds).union(proc)


In [71]:


cohort = read_parquet(spark.sqlContext, 'test9_40m.parquet')
extractorBEHRT = BEHRTextraction()
dataout = extractorBEHRT.format_behrt(fullcodes, cohort, col_entry = 'study_entry', unique_in_months=6)

In [72]:
dataout.write.parquet('test9_40m__BEHRTbasedout.parquet')
